# A* algorithm - step by step
In the following, we develop the necessary code to implement the A* algorithm in C++.

## Store a grid in your program
In order to write the A* search algorithm, I'll need a grid to search through. 

Let's start with a hard-coded grid.

In [1]:
#include <iostream>
#include <vector>
using std::cout;
using std::vector;

vector<vector<int>> board = {
    {0, 1, 0, 0, 0, 0},
    {0, 1, 0, 0, 0, 0},
    {0, 1, 0, 0, 0, 0},
    {0, 1, 0, 0, 0, 0},
    {0, 0, 0, 0, 1, 0}
};

for (vector row : board) {
    for (int value : row) {
        cout << value << " ";
    }
    cout << "\n";
}

0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 0 0 0 1 0 


## Create a function to print the board: PrintBoard
Let's put the printing of the board into a function.

In [2]:
#include <iostream>
#include <vector>
using std::cout;
using std::vector;

In [3]:
// When I try using vector instead of std::vector, the
// compiler complains about that.
// TODO: Look into this in more detail and potentially raise an issue
// at: https://github.com/jupyter-xeus/xeus-cling/issues
void PrintBoard(const std::vector<std::vector<int>> board) {
    for (std::vector row : board) {
        for (int value : row) {
            cout << value << " ";
        }
        cout << "\n";
    }
}

In [4]:
PrintBoard(board)

0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 0 0 0 1 0 


## Create a function to read the board from a file: ReadBoardFile

In [5]:
#include <iostream>
#include <fstream>

In [8]:
void ReadBoardFile(const std::string &path) {
    std::ifstream board_file(path);
    if (board_file) {
        std::string line;
        while (getline(board_file, line)) {
            std::cout << line << "\n";
        }
    }
}

In [9]:
ReadBoardFile("files/1.board")

0,1,0,0,0,0,
0,1,0,0,0,0,
0,1,0,0,0,0,
0,1,0,0,0,0,
0,0,0,0,1,0,
